In [ ]:
import pyarrow as pa
import pyarrow.flight

client = pa.flight.connect("grpc://0.0.0.0:8815")


In [ ]:
# Upload a new dataset
data_table = pa.table(
    [["Mario", "Luigi", "Peach"]],
    names=["Character"]
)
upload_descriptor = pa.flight.FlightDescriptor.for_path("characters/")
writer, _ = client.do_put(upload_descriptor, data_table.schema)
writer.write_table(data_table)
writer.close()

In [ ]:
upload_descriptor

In [ ]:
info = client.get_flight_info(upload_descriptor)

In [ ]:
info.endpoints[0].ticket

In [ ]:
reader = client.do_get(info.endpoints[0].ticket)
table = reader.read_all()
table.to_pandas()

## A more serious test with more data

In [ ]:
import pyarrow.parquet as pq
housing = pq.read_table("../housing_market/clean_export_pararius_24122021.parquet")
housing.to_pandas().head()

In [ ]:
upload_descriptor = pa.flight.FlightDescriptor.for_path("housing")
writer, _ = client.do_put(upload_descriptor, housing.schema)
writer.write_table(housing)
writer.close()

In [ ]:
endpoints = client.get_flight_info(upload_descriptor).endpoints
reader = client.do_get(endpoints[0].ticket)
table = reader.read_all()
table.to_pandas().head()

In [ ]:
endpoints[0].ticket

In [ ]:
for flight in client.list_flights():
    descriptor = flight.descriptor
    print("Path:", descriptor.path[0].decode('utf-8'), "Rows:", flight.total_records, "Size:", flight.total_bytes)
    print("=== Schema ===")
    print(flight.schema)
    print("==============")
    print("")

## DeltaServer test

In [ ]:
client = pa.flight.connect("grpc://0.0.0.0:8815")

In [ ]:
# List all Delta Tables
for flight in client.list_flights():
    descriptor = flight.descriptor
    print("Path:", descriptor.path[0].decode('utf-8'), "Rows:", flight.total_records, "Size:", flight.total_bytes)
    print("=== Schema ===")
    print(flight.schema)
    print("==============")
    print("")

In [ ]:
flight_descriptor = pa.flight.FlightDescriptor.for_path("housing/")
endpoint = client.get_flight_info(flight_descriptor).endpoints[0]

In [ ]:
table = client.do_get(endpoint.ticket).read_all()
table.to_pandas()